In [1]:
## 1. The coefficient estimates would not be unique if OLS is used to estimate them.
##    It is better to use LASSO to shrink the regression as it can also plays the role of
##    variable selection to produce a optimized combination of variables with unique coefficients.

In [2]:
## 2. f

In [3]:
## 3. f

In [4]:
## 4. The training error and the validation error are almost equal, so I would say overfitting is not the main concern.
##    However, underfitting might be a problem because both of the errors are high. Therefore, the model suffer high bias
##    rather than high variance. In this case, I would reduce the regularization parameter since I think the coefficients
##    are shrinked too much so they don't have enough weight or effect on predictor data.

In [1]:
## 5a
import boto3
import pandas as pd
import numpy as np

s3 = boto3.resource('s3')
bucket = s3.Bucket('danhtran358-data-445-bucket')
bucket_object = bucket.Object('CarPrice_Assignment.csv')
file_object = bucket_object.get()
file_stream = file_object.get('Body')

car_price = pd.read_csv(file_stream)
car_price.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [2]:
## 5b
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
## Predictors and target variables
X = car_price[['wheelbase', 'enginesize', 'compressionratio', 'horsepower', 'peakrpm', 'citympg', 'highwaympg']]
Y = car_price['price']

## List to store coefficients
lasso_coef = list()

for i in range(1000):

    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)
    
    ## Estimate lambda using cv
    lasso_cv = LassoCV(normalize = True, cv = 5).fit(X_train, Y_train)
    
    ## Building LASSO model
    lasso_md = Lasso(alpha = lasso_cv.alpha_, normalize = True).fit(X_train, Y_train)
    
    lasso_coef.append(lasso_md.coef_)

lasso_coef_df = pd.DataFrame(lasso_coef)
lasso_coef_df.head()

,0,1,2,3,4,5,6
0,235.920252,90.931602,283.277274,65.802398,1.612551,-59.911543,-0.00000
1,164.705343,97.379029,322.029890,49.710297,1.408133,-150.492660,0.00000
2,150.129434,119.515528,236.226867,49.716555,1.795407,-61.101455,-0.00000
3,220.822894,106.326685,254.190123,40.844589,1.968506,-64.514084,-42.86277
4,206.877266,90.008482,296.490464,52.197181,1.358837,-142.896181,-0.00000


In [3]:
## get the count of 0 in each column
print((lasso_coef_df == 0).sum())

0      0
1      0
2      0
3      0
4      1
5     25
6    750
dtype: int64


In [4]:
## Dropping column highwaympg
X = X.drop(columns = ['highwaympg'], axis = 1)
X.head()

,wheelbase,enginesize,compressionratio,horsepower,peakrpm,citympg
0,88.6,130,9.0,111,5000,21
1,88.6,130,9.0,111,5000,21
2,94.5,152,9.0,154,5000,19
3,99.8,109,10.0,102,5500,24
4,99.4,136,8.0,115,5500,18


In [5]:
## 5d
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

def l2_normalization(X):
    
    x_mean = np.mean(X)
    l2 = np.sqrt(sum(X**2))
    return (X - x_mean) / l2

X_train = X_train.apply(l2_normalization, axis = 1)
X_test = X_test.apply(l2_normalization, axis = 1)

In [15]:
## 5e
## Building linear model
linear_md = LinearRegression().fit(X_train, Y_train)

## Predicting on test dataset
preds1 = linear_md.predict(X_test)

mse1 = np.mean(np.power(preds1 - Y_test, 2))
mse1

12657070.581105765

In [19]:
## 5f
## list to store optimal lambdas
optimal_lambdas = list()

for i in range(100):
    
    ridge_cv = RidgeCV(alphas = [0.001, 0.01, 0.1, 1, 10, 100], cv = 5). fit(X_train, Y_train)
    
    optimal_lambdas.append(ridge_cv.alpha_)

In [20]:
## Get the frequency of each lambda
from collections import Counter
Counter(optimal_lambdas)

Counter({0.001: 100})

In [21]:
## Build model on the same dataset that built LinearRegression model
ridge_md = Ridge(alpha = 0.001).fit(X_train, Y_train)

## Predicting on test dataset
preds2 = ridge_md.predict(X_test)

mse2 = np.mean(np.power(preds2 - Y_test, 2))
mse2

13831381.875830593

In [ ]:
## 5g. I would use the first model (linear_md) as it has lower MSE, or lower error in predictions